### Import libraries

In [52]:
import pandas as pd
import numpy as np
import io
from functools import partial

In [20]:
with open('Mousotron.log', 'r') as log:
    logs = log.readlines()


In order to process the data from the log directly, i will consider the spaces in the document and identify how many are them for the column and for the data. Once i know these numbers i can effectively pull in the data.

- Date: 0 for column name, 3 spaces between the data. 
- days hour min sec: 3 for column name, 7 spaces between the data. (will use tuple notation for the next ones)
- Distance 1: (1, 1)
- Distance 2: (1, 1)
- Distance 3: (1, 1)
- keystrokes: (0,0)
- left: (0,0)
- right: (0,0)
- middle: (0,0)
- double: (0,0)
- wheel: (0,0)
- x1 btn: (1,0)
- x2 btn: (1,0)
- idle days hour min sec: (4,7)

In [113]:
from utils import spaces

cols = logs[0].split()
rows = [x.split() for x in logs[1:]]

def format_row(row, column_names=False, spaces=None):
    i_line = 0
    new_row = []
    spaces_idx = 0 if column_names else 1
    for col_spaces in spaces:
        slice_range = col_spaces[spaces_idx] + 1
        new_row.append(" ".join(row[i_line:slice_range + i_line]))
        i_line += slice_range
    return new_row

new_cols = format_row(cols, column_names=True, spaces=spaces)
df = pd.DataFrame(list(map(partial(format_row, column_names=False, spaces=spaces), rows)), columns = new_cols) # use partial to pass arguments to mapping function
df

,Date,days hour min sec,Distance 1 Distance 2 Distance 3,keystrokes,left,right,middle,double,wheel,x1 btn,x2 btn,idle days hour min sec
0,2020 abr. 07 23:32:52,0 days 13 hours 17 min 37 sec,1 km 43 m 3 cm,20590,3284,4758,15,2241,6683,0,0,0 days 7 hours 57 min 20 sec
1,2020 abr. 08 00:18:32,0 days 14 hours 3 min 17 sec,1 km 53 m 94 cm,20590,3363,4758,15,2248,6683,0,0,0 days 7 hours 57 min 33 sec
2,2020 abr. 09 00:00:12,0 days 14 hours 54 min 29 sec,0 km 663 m 26 cm,8535,2127,17,5,156,4854,0,0,0 days 0 hours 0 min 0 sec
3,2020 abr. 09 01:34:37,0 days 16 hours 28 min 54 sec,0 km 709 m 74 cm,9293,2314,17,5,167,5078,0,0,0 days 13 hours 25 min 58 sec
4,2020 abr. 10 14:05:30,0 days 2 hours 18 min 9 sec,1 km 316 m 7 cm,3935,3160,3849,1,1886,264,0,0,0 days 10 hours 24 min 27 sec
...,...,...,...,...,...,...,...,...,...,...,...,...
773,2022 nov. 01 15:55:54,26 days 16 hours 43 min 38 sec,12 km 43 m 61 cm,151297,35995,1933,137,3182,110598,0,0,19 days 19 hours 11 min 44 sec
774,2022 nov. 02 00:00:16,27 days 0 hours 46 min 13 sec,12 km 792 m 21 cm,158540,37063,1965,138,3280,114895,0,0,20 days 17 hours 46 min 28 sec
775,2022 nov. 03 00:00:57,28 days 0 hours 46 min 54 sec,13 km 891 m 48 cm,178014,40472,2235,142,3791,123208,0,0,21 days 10 hours 3 min 36 sec
776,2022 nov. 04 00:00:54,29 days 0 hours 46 min 51 sec,15 km 149 m 67 cm,187828,43436,2311,143,4052,133183,0,0,22 days 3 hours 43 min 55 sec


Dropping x1 and x2 btn rows because they don't bring in useful data.

In [114]:
df[['x1 btn', 'x2 btn']].value_counts()

x1 btn  x2 btn
0       0         513
5       10        265
dtype: int64

In [115]:
df.drop(['x1 btn', 'x2 btn'], inplace=True, axis=1)

Renaming columns

In [119]:
df.rename({
    'days hour min sec': 'Active time',
    'Distance 1 Distance 2 Distance 3': 'Distance',
    'keystrokes': 'Keystrokes',
    'left': 'Left clicks',
    'right': 'Right clicks',
    'middle': 'Middle clicks',
    'double': 'Double clicks',
    'wheel': 'Wheel turn',
    'idle days hour min sec': 'Idle time'
}, inplace=True, axis=1)


In [120]:
df

,Date,Active time,Distance,Keystrokes,Left clicks,Right clicks,Middle clicks,Double clicks,Wheel turn,Idle time
0,2020 abr. 07 23:32:52,0 days 13 hours 17 min 37 sec,1 km 43 m 3 cm,20590,3284,4758,15,2241,6683,0 days 7 hours 57 min 20 sec
1,2020 abr. 08 00:18:32,0 days 14 hours 3 min 17 sec,1 km 53 m 94 cm,20590,3363,4758,15,2248,6683,0 days 7 hours 57 min 33 sec
2,2020 abr. 09 00:00:12,0 days 14 hours 54 min 29 sec,0 km 663 m 26 cm,8535,2127,17,5,156,4854,0 days 0 hours 0 min 0 sec
3,2020 abr. 09 01:34:37,0 days 16 hours 28 min 54 sec,0 km 709 m 74 cm,9293,2314,17,5,167,5078,0 days 13 hours 25 min 58 sec
4,2020 abr. 10 14:05:30,0 days 2 hours 18 min 9 sec,1 km 316 m 7 cm,3935,3160,3849,1,1886,264,0 days 10 hours 24 min 27 sec
...,...,...,...,...,...,...,...,...,...,...
773,2022 nov. 01 15:55:54,26 days 16 hours 43 min 38 sec,12 km 43 m 61 cm,151297,35995,1933,137,3182,110598,19 days 19 hours 11 min 44 sec
774,2022 nov. 02 00:00:16,27 days 0 hours 46 min 13 sec,12 km 792 m 21 cm,158540,37063,1965,138,3280,114895,20 days 17 hours 46 min 28 sec
775,2022 nov. 03 00:00:57,28 days 0 hours 46 min 54 sec,13 km 891 m 48 cm,178014,40472,2235,142,3791,123208,21 days 10 hours 3 min 36 sec
776,2022 nov. 04 00:00:54,29 days 0 hours 46 min 51 sec,15 km 149 m 67 cm,187828,43436,2311,143,4052,133183,22 days 3 hours 43 min 55 sec


In [88]:
df.to_csv('mousotron_data.csv', index=False)

In [121]:
df = pd.read_csv('mousotron_data.csv')
df

,Date,Active time,Distance,Keystrokes,Left clicks,Right clicks,Middle clicks,Double clicks,Wheel turn,Idle time
0,2020 abr. 07 23:32:52,0 days 13 hours 17 min 37 sec,1 km 43 m 3 cm,20590,3284,4758,15,2241,6683,0 days 7 hours 57 min 20 sec
1,2020 abr. 08 00:18:32,0 days 14 hours 3 min 17 sec,1 km 53 m 94 cm,20590,3363,4758,15,2248,6683,0 days 7 hours 57 min 33 sec
2,2020 abr. 09 00:00:12,0 days 14 hours 54 min 29 sec,0 km 663 m 26 cm,8535,2127,17,5,156,4854,0 days 0 hours 0 min 0 sec
3,2020 abr. 09 01:34:37,0 days 16 hours 28 min 54 sec,0 km 709 m 74 cm,9293,2314,17,5,167,5078,0 days 13 hours 25 min 58 sec
4,2020 abr. 10 14:05:30,0 days 2 hours 18 min 9 sec,1 km 316 m 7 cm,3935,3160,3849,1,1886,264,0 days 10 hours 24 min 27 sec
...,...,...,...,...,...,...,...,...,...,...
773,2022 nov. 01 15:55:54,26 days 16 hours 43 min 38 sec,12 km 43 m 61 cm,151297,35995,1933,137,3182,110598,19 days 19 hours 11 min 44 sec
774,2022 nov. 02 00:00:16,27 days 0 hours 46 min 13 sec,12 km 792 m 21 cm,158540,37063,1965,138,3280,114895,20 days 17 hours 46 min 28 sec
775,2022 nov. 03 00:00:57,28 days 0 hours 46 min 54 sec,13 km 891 m 48 cm,178014,40472,2235,142,3791,123208,21 days 10 hours 3 min 36 sec
776,2022 nov. 04 00:00:54,29 days 0 hours 46 min 51 sec,15 km 149 m 67 cm,187828,43436,2311,143,4052,133183,22 days 3 hours 43 min 55 sec
